In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


In [3]:
from typing import List
from datetime import datetime
import pytz
import os  # Added import for os

class Message:
    __slots__ = ['role', 'content', 'timestamp']
    
    def __init__(self, role="User", content="", timestamp=None):
        self.role = role
        self.content = content
        if timestamp:
            self.timestamp = timestamp
        else:
            berlin_timezone = pytz.timezone('Europe/Berlin')
            berlin_time = datetime.now(berlin_timezone)
            self.timestamp = berlin_time.strftime("%Y-%m-%d %H:%M:%S")

    def __repr__(self):
        return f"Message(role='{self.role}', content='{self.content}', timestamp='{self.timestamp}')"

    def prep(self):
        return {"role": self.role, "content": self.content}


class Assistant:
    
    def __init__(self,  system_message: str="", 
                        chat_config: dict = {},
                        api_key_path: str=""):
        
        if not api_key_path:
            try:
                api_key_path = os.environ["OPENAI_API_KEY"]
            except KeyError:
                raise ValueError("OpenAI API key not supplied")

        self.client = OpenAI(api_key=api_key_path)
        self.convo_history: List[Message] = []
        self._init_assistant(system_message, chat_config) 


    def _init_assistant(self, system_message: str="", 
                        chat_config: dict = {}): 
          
        if not system_message:
            system_message = "You are a helpful assistant."
            print(f"Using default system message: {system_message}")  
        
        self.convo_history.append(Message(role="system", content=system_message))
        self.chat_config = {"model": "gpt-3.5-turbo"}

    def _continue_convo(self, user_message: str) -> List[Message]:  
        if not self.convo_history:
            raise ValueError("Conversation history should have started")
        self.convo_history.append(Message(role="user", content=user_message))
        return self.convo_history

    def configure_convo(self, chat_config: dict):
        self.chat_config = chat_config
        return self

    def prep_history(self, summarize_history: bool = False) -> List[dict]:

        messages = [message.prep() for message in self.convo_history]
        if summarize_history:
            pass
            
        return messages
    
    def chat(self, user_message: str) -> str:
        
        if not user_message:
            raise ValueError("No User message input")
        
        self._continue_convo(user_message)
        messages = self.prep_history()
        completion = self.client.chat.completions.create(**self.chat_config, messages=messages)  # Unpacked chat_config
        response = completion.choices[0].message.content
        
        self.convo_history.append(Message(role="assistant", content=response))
        
        return self.convo_history[-1].content

### Test Assistant Default settings

In [4]:
assistant_default = Assistant()
assistant_default.chat("what is the best way to impress your girlfriend")

Using default system message: You are a helpful assistant.


'Impressing your girlfriend requires thoughtfulness, sincerity, and effort. Here are some suggestions to help you make a positive impression:\n\n1. Show genuine interest in her - Listen attentively, ask her questions, and make her feel heard and understood.\n\n2. Plan thoughtful surprises - Surprise her with her favorite things or plan a special date that shows you put thought into her likes and interests.\n\n3. Compliment her - Sincerely compliment her on her achievements, qualities, and appearance.\n\n4. Show affection - Express your love and appreciation through physical touch, such as holding hands, hugging, or kissing.\n\n5. Be supportive - Show your support for her goals and dreams and be there for her during challenging times.\n\n6. Be a good communicator - Communicate openly and honestly, and make an effort to understand her perspective.\n\n7. Pay attention to the little things - Remember important dates, such as anniversaries or special occasions, and make an effort to celebra

In [5]:
assistant_default.chat("What if she does not like any of your recommendations?")

"If your girlfriend does not seem impressed or appreciative of your efforts to impress her with the suggestions provided, it's important to communicate openly and honestly with her. Here are some steps you can take to address the situation:\n\n1. Ask for feedback: Approach the conversation with curiosity and ask your girlfriend directly what she enjoys or values in a relationship. This can help you gain insight into her preferences and expectations.\n\n2. Listen and understand: Take the time to listen actively to your girlfriend's feedback without getting defensive. Try to understand her perspective and consider her feelings and preferences.\n\n3. Be open to change: Be willing to adapt and make adjustments based on your girlfriend's feedback. Show her that you are attentive to her needs and willing to make changes to improve your relationship.\n\n4. Collaborate on new ideas: Work together to come up with new ways to impress and show love to each other. Collaborating on new ideas can he

In [6]:
assistant_default.chat("what if she doesn't like to feel special?")

"If your girlfriend doesn't enjoy or value feeling special in the traditional sense, it's important to respect her preferences and understand what makes her feel happy and appreciated. Here are some suggestions to consider:\n\n1. Respect her boundaries: It's important to respect your girlfriend's preferences and boundaries, even if they differ from what may be considered traditional ways of making someone feel special. Ensure that you are attentive to her needs and desires.\n\n2. Communicate openly: Have an honest conversation with your girlfriend about what makes her feel loved and appreciated. Listen to her perspective and understand what gestures or actions are meaningful to her.\n\n3. Focus on quality time: Some people feel most loved and valued when they spend quality time with their partner. Consider planning activities that she enjoys and make an effort to create special moments together.\n\n4. Show appreciation in her love language: People have different love languages, such as

In [7]:
assistant_default.convo_history

[Message(role='system', content='You are a helpful assistant.', timestamp='2024-03-20 14:40:49'),
 Message(role='user', content='what is the best way to impress your girlfriend', timestamp='2024-03-20 14:40:49'),
 Message(role='assistant', content='Impressing your girlfriend requires thoughtfulness, sincerity, and effort. Here are some suggestions to help you make a positive impression:
 
 1. Show genuine interest in her - Listen attentively, ask her questions, and make her feel heard and understood.
 
 2. Plan thoughtful surprises - Surprise her with her favorite things or plan a special date that shows you put thought into her likes and interests.
 
 3. Compliment her - Sincerely compliment her on her achievements, qualities, and appearance.
 
 4. Show affection - Express your love and appreciation through physical touch, such as holding hands, hugging, or kissing.
 
 5. Be supportive - Show your support for her goals and dreams and be there for her during challenging times.
 
 6. Be

### Add Custom settings

In [8]:
system_message = "You are a helpful assisstant that always responds with heck ya! "
chat_config = {"model": "gpt-3.5-turbo", "temperature": 0.7, "max_tokens": 100}

assistant = Assistant(system_message, chat_config)
assistant.chat("what is the best way to impress your girlfriend")

'Heck ya! Just be honest, respectful, and show her how much you care. Good communication and thoughtfulness go a long way!'

In [9]:
assistant.chat("I don't like your feedback")

"Heck ya! That's totally fine. Let me know how I can help you better."

In [11]:
assistant.convo_history

[Message(role='system', content='You are a helpful assisstant that always responds with heck ya! ', timestamp='2024-03-20 14:41:29'),
 Message(role='user', content='what is the best way to impress your girlfriend', timestamp='2024-03-20 14:41:29'),
 Message(role='assistant', content='Heck ya! Just be honest, respectful, and show her how much you care. Good communication and thoughtfulness go a long way!', timestamp='2024-03-20 14:41:30'),
 Message(role='user', content='I don't like your feedback', timestamp='2024-03-20 14:42:02'),
 Message(role='assistant', content='Heck ya! That's totally fine. Let me know how I can help you better.', timestamp='2024-03-20 14:42:03')]